<a href="https://colab.research.google.com/github/BruunoRangel/BruunoRangel/blob/main/Aula_10_Exerc%C3%ADcio_09_Template_Bruno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nome = "Bruno Rangel Balbino dos Santos "
print(f'Meu nome é {nome}')

Meu nome é Bruno Rangel Balbino dos Santos 


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [ ]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 45.9 MB/s 
     |████████████████████████████████| 86 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 30.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [ ]:
# Check which GPU we are using
!nvidia-smi

Wed Jun  8 20:20:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
from typing import List
from tqdm.notebook import tqdm


def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        self.max_seq_length = max_seq_length
        self.examples = []
        for text in texts:
         
          tokenized_text = tokenize(f'[CLS] {text}', tokenizer) # insere o vetor [101] 

          tokenized_text += [tokenizer.vocab['[PAD]']] * max(0, 1 + max_seq_length - len(tokenized_text))
          
          for i in range(0, len(tokenized_text) - 1, max_seq_length):
            if i + max_seq_length < len(tokenized_text):
              self.examples.append(tokenized_text[i: i + max_seq_length + 1])
            else:
              self.examples.append(tokenized_text[-max_seq_length - 1:])
        self.examples = torch.LongTensor(self.examples)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        x_y = self.examples[idx]
        
        return x_y[:-1], x_y[1:]

## Testando se a implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Passou no assert de tamanho do dataset.
Passou no assert de dataset.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2022-06-08 20:20:22--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.183.128, 209.85.147.128, 142.250.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.183.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G   146MB/s    in 8.6s    

2022-06-08 20:20:30 (137 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



In [ ]:
# Load datasets
max_seq_length = 9

train_examples =  50000
valid_examples = 1000
test_examples = 1000

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.
Truncating to 52000 lines.


In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 6186062
valid examples: 136756
test examples: 125212


In [ ]:
class MultiHeadAttentionNN(nn.Module):

  def __init__(self, max_seq_length, embedding_dim, num_heads, padding_id):
    super().__init__() 

    self.H = num_heads
    self.D = embedding_dim
    self.max_seq_length = max_seq_length

    self.W_q = torch.nn.Linear(embedding_dim, embedding_dim, bias = False)  
    self.W_k = torch.nn.Linear(embedding_dim, embedding_dim, bias = False) 
    self.W_v = torch.nn.Linear(embedding_dim, embedding_dim, bias = False)  
    self.mask = torch.tril(torch.ones(max_seq_length, max_seq_length)).unsqueeze(0).to(device)
    
    
    
  def scaled_dot_product(self, q, k, v, mask):
    # q=k=v.shape = B, H, L, D/H
    # mask.shape = B, L
    scores = torch.matmul(q, torch.transpose(k, -2, -1)).to(device)  
    scores = scores.masked_fill(mask.unsqueeze(1) == 0, -1e9)
    scores /= (self.D ** -0.5)  
    probs = torch.nn.functional.softmax(scores, dim=-2) 
    E = torch.matmul(probs, v)  
    
    return E.transpose(1, 2)

  def forward(self, inputs, mask):                   
    # x.shape = B, L, D
    residual = inputs

    B = inputs.shape[0]        
    L = inputs.shape[1]          
    H = self.H         
    D = self.D       

    q = self.W_q(inputs).reshape(B, L, H, int(D/H))
    k = self.W_k(inputs).reshape(B, L, H, int(D/H))
    v = self.W_v(inputs).reshape(B, L, H, int(D/H))
    
    
    # new_inputs.shape = B, H, L, D/H
    q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)
    

    new_inputs = self.scaled_dot_product(q, k, v, mask)   
    
    
    new_inputs = new_inputs.reshape(B, L, D) 
    

    new_inputs += residual
    

    return new_inputs

In [ ]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        super().__init__()
        self.pad_token_id = pad_token_id
        self.max_seq_length = max_seq_length
        self.embedding = torch.nn.Embedding(vocab_size, dim, padding_idx=pad_token_id)
        self.positional_embeddings = torch.nn.Embedding(vocab_size, dim, padding_idx=pad_token_id)
        self.heads = MultiHeadAttentionNN(max_seq_length, dim, n_layers, pad_token_id)
        self.W_o = torch.nn.Linear(dim, dim, bias = False)
        self.layer_norm = nn.LayerNorm(dim)
        self.linear01 = torch.nn.Linear(dim, dim*2)
        self.relu = torch.nn.ReLU()
        self.linear02 = torch.nn.Linear(dim*2, vocab_size, bias=False)
     

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        batch_size = inputs.shape[0]

        mask = torch.tril(torch.ones(batch_size, self.max_seq_length, self.max_seq_length)).to(device)
        mask = mask.masked_fill(inputs.unsqueeze(1) == self.pad_token_id, 0)
        


        embds = self.embedding(inputs) + self.positional_embeddings(inputs)  # shape = B, L, D
        
        embds = self.heads.forward(embds, mask)
        embds = self.W_o(embds)
        embds = self.layer_norm(embds)
        

        embds1 = self.heads.forward(embds, mask)
        embds1 = self.W_o(embds1)
        embds1 = self.layer_norm(embds1)
       

        output = self.linear01(embds1)
        output = self.relu(output)
        output = self.linear02(output)
        #print(output.shape)

        return output

## Teste o modelo com um exemplo

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 7652096


## Assert da Perplexidade


In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30241
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:
max_examples = 16_000_000
eval_every_steps = 10000
lr = 3e-4



model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=64)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 30380.35, valid ppl: 30437.08
10000 steps; 640000 examples so far; train ppl: 895.23, valid ppl: 581.40
20000 steps; 1280000 examples so far; train ppl: 475.49, valid ppl: 400.91
30000 steps; 1920000 examples so far; train ppl: 360.81, valid ppl: 328.23
40000 steps; 2560000 examples so far; train ppl: 311.67, valid ppl: 294.29
50000 steps; 3200000 examples so far; train ppl: 285.69, valid ppl: 275.16
60000 steps; 3840000 examples so far; train ppl: 270.66, valid ppl: 261.96
70000 steps; 4480000 examples so far; train ppl: 259.67, valid ppl: 252.95
80000 steps; 5120000 examples so far; train ppl: 250.45, valid ppl: 244.77
90000 steps; 5760000 examples so far; train ppl: 243.55, valid ppl: 239.38
100000 steps; 6400000 examples so far; train ppl: 236.44, valid ppl: 234.52
110000 steps; 7040000 examples so far; train ppl: 228.98, valid ppl: 230.76
120000 steps; 7680000 examples so far; train ppl: 226.36, valid ppl: 227.01
130000 steps; 8320000 example

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 206.531683260952


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz parte
Eu gosto de comer pizza pois me faz parte da
Eu gosto de comer pizza pois me faz parte da sociedade
Eu gosto de comer pizza pois me faz parte da sociedade.
Eu gosto de comer pizza pois me faz parte da sociedade. O
Eu gosto de comer pizza pois me faz parte da sociedade. O que
Eu gosto de comer pizza pois me faz parte da sociedade. O que o
Eu gosto de comer pizza pois me faz parte da sociedade. O que o que
Eu gosto de comer pizza pois me faz parte da sociedade. O que o que se
Eu gosto de comer pizza pois me faz parte da sociedade. O que o que se,
Eu gosto de comer pizza pois me faz parte da sociedade. O que o que se, a
Eu gosto de comer pizza pois me faz parte da sociedade. O que o que se, a partir
Eu gosto de comer pizza pois me faz parte da sociedade. O que o que se, a partir da
Eu gosto de comer pizza pois me faz parte da sociedade. O que o que se, a partir da sociedade
Eu gosto de comer pizza pois me faz parte da sociedade. O que o que se, a 

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.